# Schema and Data Exploration

This notebook demonstrates how to inspect the SQLite database schema and sample data.

In [10]:
import sqlite3
import pandas as pd
import os

# Connect to the database
db_path = "../inventory_v2.db"
conn = sqlite3.connect(db_path)

def list_tables(conn):
    query = "SELECT name FROM sqlite_master WHERE type='table';"
    return pd.read_sql(query, conn)

print("Tables in Database:")
display(list_tables(conn))

Tables in Database:


,name
0,sqlite_sequence
1,orders_clean
2,inventory_clean
3,customers_clean
4,transactions_clean
5,channel_kpis
6,channels_raw
7,inventory_snapshots_raw
8,customers
9,orders_raw


In [11]:
# Inspect Raw Orders
print("Raw Orders Sample:")
display(pd.read_sql("SELECT * FROM orders_raw LIMIT 5", conn))

Raw Orders Sample:


,order_id,customer_id,order_date,channel_id,total_amount,status
0,ORD-PmVK8223,CUST-06712,2025-11-20 09:58:23,CH_RET,249.53,PENDING
1,ORD-VZyt0327,CUST-12329,2025-11-19 05:33:09,CH_RET,1081.76,COMPLETED
2,ORD-ZMeb9675,CUST-76304,2025-11-03 22:06:11,CH_SHP,971.04,PENDING
3,ORD-CcMm3611,CUST-30169,2025-11-06 07:09:21,CH_SHP,179.36,SHIPPED
4,ORD-vefO7257,CUST-62412,2025-11-27 13:30:12,CH_AMZ,563.37,SHIPPED


In [26]:
import numpy as np

In [31]:
table_df = list_tables(conn)

#convert tables to list
table_list = table_df.values.tolist()
#flatten the list
table_list = [item for sublist in table_list for item in sublist]

for table in table_list[1:-1]:
    print("Table: {}".format(table))
    display(pd.read_sql("SELECT * FROM {} LIMIT 5".format(table), conn))

Table: channels_raw


,channel_id,channel_name,platform,commission_rate
0,CH_AMZ,Amazon,Amazon,0.15
1,CH_SHP,Shopify Store,Shopify,0.03
2,CH_RET,Retail POS,Physical,0.00
3,CH_EBY,eBay,eBay,0.10


Table: inventory_snapshots_raw


,sku,warehouse_id,quantity_on_hand,snapshot_date
0,SKU-TX004,WH_NY,28,2025-10-28
1,SKU-TX004,WH_CA,67,2025-10-28
2,SKU-TX004,WH_TX,86,2025-10-28
3,SKU-Ey537,WH_NY,24,2025-10-28
4,SKU-Ey537,WH_CA,30,2025-10-28


Table: customers


...

,sku,warehouse_id,quantity_on_hand,snapshot_date,processing_timestamp
0,SKU-Gk370,WH_NY,81,2025-10-29,2025-11-27 18:20:25.228083
1,SKU-zI085,WH_NY,11,2025-10-29,2025-11-27 18:20:25.228083
2,SKU-cw888,WH_TX,86,2025-10-30,2025-11-27 18:20:25.228083
3,SKU-Db118,WH_CA,83,2025-10-28,2025-11-27 18:20:25.228083
4,SKU-VA832,WH_CA,77,2025-10-28,2025-11-27 18:20:25.228083


Table: customers_clean


,customer_id,first_name,last_name,email,address,city,state,zip_code,signup_date,processing_timestamp
0,CUST-00915,Lauren,Skinner,jesseshah@example.net,7270 Willis Parkway,Brianchester,MA,27453,2025-04-15,2025-11-27 18:20:38.369707
1,CUST-00975,Curtis,Thomas,jeffreywhitney@example.net,680 Martin Port Apt. 818,Lake Thomaschester,PA,73555,2025-09-24,2025-11-27 18:20:38.369707
2,CUST-01122,Crystal,Clark,flowersangela@example.org,9642 Javier Inlet Suite 203,Lake Sean,TX,78414,2025-03-23,2025-11-27 18:20:38.369707
3,CUST-01367,David,King,aguilarspencer@example.net,07790 Leslie Ways Suite 339,Lake Kelseyton,OR,02990,2025-10-08,2025-11-27 18:20:38.369707
4,CUST-01386,Nancy,Coleman,diazshane@example.org,69978 Tracey Tunnel,Stephensview,CO,02443,2025-08-18,2025-11-27 18:20:38.369707


Table: transactions_clean


,transaction_id,order_id,payment_method,amount,transaction_date,status,processing_timestamp
0,TXN-AFIg8331,ORD-IdYX4981,Credit Card,385.42,2025-10-28,SUCCESS,2025-11-27 18:20:49.878528
1,TXN-ATwM2888,ORD-PNuS7682,Credit Card,241.05,2025-11-20,SUCCESS,2025-11-27 18:20:49.878528
2,TXN-AdGJ6267,ORD-RQwN7612,PayPal,1324.19,2025-10-30,SUCCESS,2025-11-27 18:20:49.878528
3,TXN-AdMm8891,ORD-doDD7019,PayPal,936.09,2025-10-29,SUCCESS,2025-11-27 18:20:49.878528
4,TXN-ArbL8473,ORD-ILou5178,Credit Card,1248.79,2025-11-19,SUCCESS,2025-11-27 18:20:49.878528
